In [1]:
import torch
from datasets import Dataset

# Check if GPU is available
if torch.cuda.is_available():
    # Get the number of available GPUs
    num_gpus = torch.cuda.device_count()
    print(f"Number of available GPUs: {num_gpus}")
    for gpu_id in range(num_gpus):
        print(f"GPU {gpu_id}: {torch.cuda.get_device_name(gpu_id)}")
else:
    print("No GPU available.")

c:\Users\Aniket Konkar\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of available GPUs: 1
GPU 0: NVIDIA GeForce RTX 3070


The below code is fetching the prepared data and providing it as input to train the Vision Transformer mode. I referenced the official Hugging Face documentation to fetch the pretrained model and then train it for my usecase. Documentation Link: https://huggingface.co/docs/transformers/en/model_doc/vit

In [2]:
ds = Dataset.load_from_disk("dataset/train_dataset_for_vit")

In [3]:
import pickle

# File path to the pickle file
id2label_file_path = "dataset/id2label.pkl"
label2id_file_path = "dataset/label2id.pkl"

# Load the object from the pickle file
with open(id2label_file_path, "rb") as f:
    id2label = pickle.load(f)
    
# Load the object from the pickle file
with open(label2id_file_path, "rb") as f:
    label2id = pickle.load(f)

In [4]:
dataset = ds.train_test_split(test_size=0.1, shuffle=True)
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'image_file_path', 'labels'],
        num_rows: 4088
    })
    test: Dataset({
        features: ['image', 'image_file_path', 'labels'],
        num_rows: 455
    })
})

In [5]:
from transformers import ViTImageProcessor

model_name_or_path = 'google/vit-base-patch16-224-in21k'
processor = ViTImageProcessor.from_pretrained(model_name_or_path)

In [6]:
def process_example(example):
    inputs = processor(example['image'], return_tensors='pt')
    inputs['labels'] = example['labels']
    return inputs

In [7]:
process_example(dataset['train'][0])

{'pixel_values': tensor([[[[-0.4196, -0.4588, -0.4196,  ..., -0.4667, -0.4902, -0.4667],
          [-0.4510, -0.4824, -0.4588,  ..., -0.4980, -0.4902, -0.4588],
          [-0.4588, -0.4353, -0.4118,  ..., -0.4667, -0.4431, -0.4431],
          ...,
          [-0.3569, -0.4196, -0.3882,  ..., -0.3961, -0.4196, -0.4667],
          [-0.3647, -0.4118, -0.4431,  ..., -0.4275, -0.4196, -0.4196],
          [-0.4039, -0.4118, -0.3882,  ..., -0.4431, -0.4353, -0.4824]],

         [[-0.4667, -0.5137, -0.4902,  ..., -0.5216, -0.5451, -0.5216],
          [-0.4980, -0.5373, -0.5294,  ..., -0.5529, -0.5451, -0.5137],
          [-0.5059, -0.4902, -0.4824,  ..., -0.5216, -0.4980, -0.4980],
          ...,
          [-0.4118, -0.4745, -0.4431,  ..., -0.4353, -0.4510, -0.4980],
          [-0.4196, -0.4667, -0.4980,  ..., -0.4667, -0.4510, -0.4510],
          [-0.4588, -0.4667, -0.4431,  ..., -0.4824, -0.4667, -0.5137]],

         [[-0.5529, -0.5922, -0.5686,  ..., -0.6549, -0.6784, -0.6549],
          [-0

In [8]:
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = processor([x for x in example_batch['image']], return_tensors='pt')

    # Don't forget to include the labels!
    inputs['labels'] = example_batch['labels']
    return inputs

In [9]:
prepared_ds = dataset.with_transform(transform)

In [10]:
prepared_ds['train'][0]['pixel_values'].shape

torch.Size([3, 224, 224])

In [11]:
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

In [12]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

C:\Users\Aniket Konkar\AppData\Local\Temp\ipykernel_2344\2425197347.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
c:\Users\Aniket Konkar\AppData\Local\Programs\Python\Python312\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [13]:
from transformers import ViTForImageClassification

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    device_map="auto",
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vit-base-beans",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=21,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=40,
  learning_rate=0.002,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='wandb',
  load_best_model_at_end=True,
)

In [15]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["test"], # its validation dataset
    tokenizer=processor,
)

The below code snippet is where the actual model training happens. I have trained multiple times to try and get the best validation accuracy and make sure my model doesn't overfit. I came to the conclusion that training for 21 epochs for this usecase and configurations seems to work the best.

In [16]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: inspiration601 (aniketkonkar). Use `wandb login --relogin` to force relogin


  1%|          | 41/5376 [00:12<14:53,  5.97it/s] 

{'loss': 6.1129, 'grad_norm': 0.7761141657829285, 'learning_rate': 0.0019851190476190476, 'epoch': 0.16}


  2%|▏         | 81/5376 [00:19<15:01,  5.87it/s]

{'loss': 6.0562, 'grad_norm': 0.6475346684455872, 'learning_rate': 0.001970238095238095, 'epoch': 0.31}


...

{'loss': 5.3847, 'grad_norm': 1.2946176528930664, 'learning_rate': 2.306547619047619e-05, 'epoch': 20.78}


100%|█████████▉| 5361/5376 [19:08<00:02,  6.03it/s]

{'loss': 5.3668, 'grad_norm': 1.496396780014038, 'learning_rate': 8.18452380952381e-06, 'epoch': 20.94}


100%|██████████| 5376/5376 [19:11<00:00,  4.67it/s]

{'train_runtime': 1155.0712, 'train_samples_per_second': 74.323, 'train_steps_per_second': 4.654, 'train_loss': 5.683738708496094, 'epoch': 21.0}
***** train metrics *****
  epoch                    =         21.0
  total_flos               = 6220364240GF
  train_loss               =       5.6837
  train_runtime            =   0:19:15.07
  train_samples_per_second =       74.323
  train_steps_per_second   =        4.654
